In [1]:
import glob
import gzip
import itertools
import csv
import io
import pickle
import hashlib
import pandas as pd
import numpy as np

# WMT16 system-level data

### Checksums

In [2]:
mine = 'c983b60fa311b60c04c0293aaf1a2bc4'
yours = hashlib.md5(open("data/downloads/wmt16-submitted-data-v2.tgz", 'rb').read()).hexdigest()
print(mine + '\n' + yours)
print(mine == yours)


mine = '2acd4f1d8fcc07115cc06bcaed4ff236'
yours = hashlib.md5(open("data/downloads/wmt16-metrics-results.tar.gz", 'rb').read()).hexdigest()
print(mine + '\n' + yours)
print(mine == yours)

# unzip to data/
if False:
    os.system('tar -xvf data/downloads/wmt16-submitted-data-v2.tgz -p data/')
    os.system('tar -xvf data/downloads/wmt16-metrics-results.tar.gz -p data/')

c983b60fa311b60c04c0293aaf1a2bc4
c983b60fa311b60c04c0293aaf1a2bc4
True
2acd4f1d8fcc07115cc06bcaed4ff236
2acd4f1d8fcc07115cc06bcaed4ff236
True


### Aggregate system-level data

In [3]:
da_files = [ i for i in glob.glob('data/wmt16-metrics-results/sys-level-results/standard/results-official/newstest2016*') if not 'noDA' in i ]
lp_df = []

for file in da_files:
    lp = file[-8:-4]
    lp = lp[0:2] + '-' + lp[2:]
    
    df = pd.read_csv(file, delimiter=' ')
    df['lp'] = lp
    
    lp_df.append(df)
sys_scores_da = pd.concat(lp_df)

/home/nlg-05/jtwei/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


Rename columns:

In [4]:
replace = {'DA':'score', 'MT':'system', 'mtevalBLEU':'BLEU', 'mtevalNIST':'NIST', 'mosesCDER':'CDER', 'mosesPER':'PER', 'mosesWER':'WER'}
sys_scores_da.columns = [ replace[i] if i in replace else i for i in sys_scores_da.columns ]

sys_scores_da = sys_scores_da[['lp', 'system', 'BLEU', 'NIST', 'CDER', 'PER', 'WER', 'TER', 'score']]
sys_scores_da

,lp,system,BLEU,NIST,CDER,PER,WER,TER,score
0,cs-en,online-B,0.2916,8.1473,0.4950,0.6103,0.4454,0.4844,0.050822
1,cs-en,uedin-nmt,0.3297,8.2267,0.5265,0.6332,0.4640,0.4971,0.207242
2,cs-en,PJATK,0.2954,7.8778,0.4913,0.6193,0.4263,0.4625,-0.024469
3,cs-en,online-A,0.2818,7.6764,0.4710,0.5907,0.3962,0.4409,0.000170
4,cs-en,jhu-pbmt,0.3178,8.1369,0.5124,0.6218,0.4434,0.4815,0.101488
...,...,...,...,...,...,...,...,...,...
2,ro-en,online-B,0.3952,9.3990,0.5859,0.6858,0.5437,0.5751,0.128884
3,ro-en,online-A,0.3291,8.1913,0.5227,0.6337,0.4527,0.4913,-0.012314
4,ro-en,jhu-pbmt,0.3342,8.2017,0.5366,0.6420,0.4727,0.5055,-0.159592
5,ro-en,uedin-syntax,0.3466,8.4785,0.5439,0.6561,0.4776,0.5137,-0.000472


### WMT16 system-level data (all/raw)

In [5]:
lp_df = []

for file in glob.glob('data/wmt-human-evaluation/da-human-judgments/ad-seg-scores-*.csv'):
    lp = file[-9:-4]
    
    df = pd.read_csv(file, delimiter=' ')
    df['lp'] = [lp] * len(df)
    
    lp_df.append(df)
raw_seg_scores_da = pd.concat(lp_df)

In [6]:
N = raw_seg_scores_da.groupby(['lp', 'SYS'], as_index=False).count()['N']
raw_sys_scores_da = raw_seg_scores_da.groupby(['lp', 'SYS'], as_index=False).mean()
raw_sys_scores_da['N'] = N

raw_sys_scores_da.columns = ['lp', 'system', 'sid', 'raw_score', 'score', 'N', '5']
raw_sys_scores_da = raw_sys_scores_da[['lp', 'system', 'raw_score', 'score', 'N']]
raw_sys_scores_da

,lp,system,raw_score,score,N
0,cs-en,PJATK,68.954545,-0.024469,2464
1,cs-en,cu-mergedtrees,55.783751,-0.502992,2437
2,cs-en,jhu-pbmt,72.568873,0.101488,2432
3,cs-en,online-A,69.455488,0.000170,2460
4,cs-en,online-B,70.844961,0.050822,2451
...,...,...,...,...,...
58,tr-en,jhu-syntax,40.803632,-0.363599,1762
59,tr-en,online-A,52.190503,0.002009,1790
60,tr-en,online-B,57.128603,0.163079,1804
61,tr-en,online-G,54.964850,0.108645,1835


In [7]:
sys_scores_da = sys_scores_da.merge(raw_sys_scores_da[['lp','system','raw_score']])
sys_scores_da[['lp','raw_score', 'score']].groupby('lp').corr('pearson')

raw_score     score
lp                                  
cs-en raw_score   1.000000  0.999818
      score       0.999818  1.000000
de-en raw_score   1.000000  0.999354
      score       0.999354  1.000000
en-ru raw_score   1.000000  0.993851
      score       0.993851  1.000000
fi-en raw_score   1.000000  0.999598
      score       0.999598  1.000000
ro-en raw_score   1.000000  0.998583
      score       0.998583  1.000000
ru-en raw_score   1.000000  0.996735
      score       0.996735  1.000000
tr-en raw_score   1.000000  0.999315
      score       0.999315  1.000000

### WMT16 (src, ref, out)

In [8]:
# srcs and refs
srcs, refs, lps, sids = [], [], [], []
for lp in sys_scores_da.lp.unique():
    fr, to = lp[:2], lp[3:]
    
    refs_ = list(open('data/wmt16-submitted-data/txt/references/newstest2016-%s%s-ref.%s' % (fr, to, to)))
    srcs_ = list(open('data/wmt16-submitted-data/txt/sources/newstest2016-%s%s-src.%s' % (fr, to, fr)))
    sids_ = list(range(1, len(refs_)+1))
    refs.extend(refs_)
    srcs.extend(srcs_)
    sids.extend(sids_)
    
    assert(len(refs_) == len(srcs_))
    lps.extend([lp]*len(refs_))
    
df = pd.DataFrame({'reference' : refs, 'source':srcs, 'lp': lps, 'SID': sids})
print('# of entries before merge: %d' % len(raw_seg_scores_da))
raw_seg_scores_da = raw_seg_scores_da.merge(df, on=['lp','SID'], how='inner')
print('# of entries after merge: %d' % len(raw_seg_scores_da))
print('These two should be equal.')

# of entries before merge: 141905
# of entries after merge: 141905
These two should be equal.


In [9]:
# outs
lps, outs, sids, syss = [], [], [], []
for file in glob.glob('data/wmt16-submitted-data/txt/system-outputs/newstest2016/*/*'):
    lp = file.split('.')[-1]
    system = file.split('.')[-3]
    
    outs_ = list(open(file, 'rt'))
    sids_ = list(range(1, len(outs_)+1))
    lps_ = len(outs_) * [lp]
    syss_ = len(outs_) * [system]
    
    outs.extend(outs_)
    sids.extend(sids_)
    lps.extend(lps_)
    syss.extend(syss_)

df = pd.DataFrame({'lp': lps, 'output':outs, 'SID': sids, 'SYS': syss})
print('# of entries before merge: %d' % len(raw_seg_scores_da))
raw_seg_scores_da = raw_seg_scores_da.merge(df, on=['lp','SID', 'SYS'], how='inner')
print('# of entries after merge: %d' % len(raw_seg_scores_da))
print('These two should be equal.')

# of entries before merge: 141905
# of entries after merge: 141905
These two should be equal.


In [10]:
raw_seg_scores_da.columns = ['system', 'sid', 'raw_score', 'score', 'N', 'nan', 'lp', 'reference', 'source', 'output']
raw_seg_scores_da = raw_seg_scores_da[['system', 'sid', 'raw_score', 'score', 'N', 'lp', 'reference', 'source', 'output']]

### Validate correlations

In [11]:
sys_scores_da.groupby('lp') \
    ['lp'] \
    .count()

lp
cs-en     6
de-en    10
en-ru    12
fi-en     9
ro-en     7
ru-en    10
tr-en     8
Name: lp, dtype: int64

In [12]:
sys_scores_da[sys_scores_da.lp.str.endswith('en')] \
    .groupby('lp') \
    .corr()[6::8] \
    .round(3) \
    .T \
    .sort_index()

lp,cs-en,de-en,fi-en,ro-en,ru-en,tr-en
,score,score,score,score,score,score
BLEU,0.989,0.808,0.864,0.840,0.837,0.895
CDER,0.988,0.827,0.860,0.800,0.855,0.826
NIST,0.978,0.801,0.929,0.807,0.854,0.938
PER,0.970,0.730,0.767,0.748,0.887,0.940
TER,0.969,0.834,0.846,0.793,0.847,0.788
WER,0.967,0.822,0.768,0.762,0.837,0.651
raw_score,1.000,0.999,1.000,0.999,0.997,0.999
score,1.000,1.000,1.000,1.000,1.000,1.000


In [13]:
sys_scores_da[~sys_scores_da.lp.str.endswith('en')] \
    .groupby('lp') \
    .corr()[6::8] \
    .round(3) \
    .T \
    .sort_index()

lp,en-ru
,score
BLEU,0.838
CDER,0.874
NIST,0.897
PER,0.854
TER,0.879
WER,0.876
raw_score,0.994
score,1.000


### Pickle

In [14]:
pickle.dump(raw_seg_scores_da, open('data/pickles/wmt16-sys_level-all.pkl', 'wb'))
pickle.dump(sys_scores_da, open('data/pickles/wmt16-sys_level-agg.pkl', 'wb'))

# WMT16 segment-level data

In [15]:
da_files = [ i for i in glob.glob('data/wmt16-metrics-results/seg-level-results/da-results/metrics.*.csv') if not 'noDA' in i ]
lp_df = []

for file in da_files:
    lp = file[-9:-4]
    
    df = pd.read_csv(file, delimiter=' ')
    df['lp'] = lp
    lp_df.append(df)
    
    df['source'] = list(open('data/wmt16-metrics-results/seg-level-results/da-results/src.%s' % lp, 'rt'))
    df['output'] = list(open('data/wmt16-metrics-results/seg-level-results/da-results/snt.%s' % lp, 'rt'))
    df['reference'] = list(open('data/wmt16-metrics-results/seg-level-results/da-results/ref.%s' % lp, 'rt'))
    
seg_scores_da = pd.concat(lp_df)

/home/nlg-05/jtwei/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [16]:
seg_scores_da.groupby('lp').count()[['SID']]

,SID
lp,
cs-en,560
de-en,560
en-ru,560
fi-en,560
ro-en,560
ru-en,560
tr-en,560


Rename columns:

In [17]:
replace = {'HUMAN.Z':'score', 'SID':'sid', 'MT':'system', 'mtevalBLEU':'BLEU', 'mtevalNIST':'NIST', 'mosesCDER':'CDER', 'mosesPER':'PER', 'mosesWER':'WER'}
seg_scores_da.columns = [ replace[i] if i in replace else i for i in seg_scores_da.columns ]
seg_scores_da = seg_scores_da[['lp', 'system', 'sid', 'source', 'reference', 'output', 'score']]
seg_scores_da

,lp,system,sid,source,reference,output,score
0,de-en,jhu-syntax,677,"Er fügte hinzu, er wünsche sich, dass das Unte...","He added that he wants the Corporation to ""fin...","He added that he hoped the company ""is a hit f...",0.691142
1,de-en,jhu-pbmt,1092,Am Mittwochmorgen hätten sich dort noch 40 Flü...,On Wednesday morning another 40 refugees were ...,On Wednesday morning there still 40 refugees w...,-0.762188
2,de-en,online-B,1375,"Die Hearts führten 2:1, als Paterson vom Feld ...",Hearts were leading 2-1 when Paterson was dism...,"The Hearts Verteidigers led 2: 1, was sent fro...",-0.599309
3,de-en,online-G,616,"Und bei jeder Puppe, die aussieht wie er und ü...",And for every doll that looks like him and pas...,And for every doll that looks like him and goe...,-0.305292
4,de-en,jhu-syntax,679,"Gyorgy Bakondi, Landes-Sicherheitsberater für ...","Gyorgy Bakondi, homeland security adviser to O...","Gyorgy Bakondi, country security adviser for O...",0.362249
...,...,...,...,...,...,...,...
555,cs-en,uedin-nmt,1769,U většího modelu je pak nejtěžším použitým prv...,"In the larger model, the largest component use...",To a larger model is heaviest used element bat...,0.221244
556,cs-en,cu-mergedtrees,2099,"Konopný olej není nijak chuťově výrazný, ale s...","While it doesn't have a strong taste, hemp oil...","oil is not a distinctive flavour, but a great ...",-1.424952
557,cs-en,cu-mergedtrees,1330,Ale přichází pocity frustrace.\n,But frustrations were on the rise.\n,But the feelings of frustration.\n,-1.107383
558,cs-en,jhu-pbmt,519,"Tento názor řekl v roce 2010 a nyní chce, aby ...","He made that point in 2010, now he wants every...",He said that view in 2010 and now wants all te...,0.111809


### Pickle

In [18]:
pickle.dump(seg_scores_da, open('data/pickles/wmt16-seg_level-agg.pkl', 'wb'))